In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install keras_preprocessing

In [ ]:
from keras.utils import to_categorical
from keras_preprocessing.image import load_img
from keras.models import Sequential
from keras.applications import MobileNetV2, ResNet152, VGG16, EfficientNetB0, InceptionV3
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization,AveragePooling2D
import os
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import regularizers
import tensorflow as tf

temp_dir = []
sqlen = 289

def createdataframe(dir):
    image_paths = []

    labels = []
    for label in os.listdir(dir):
        for imagename in os.listdir(os.path.join(dir, label)):
            image_paths.append(os.path.join(dir, label, imagename))
            labels.append(label)
        print(label, "completed")
    return image_paths, labels

def extractrain_features(images):
    features = []

    for image in tqdm(images):
        img = load_img(image, target_size=(sqlen, sqlen))
        img = np.array(img)
        features.append(img)

    features = np.array(features)
    features = features.reshape(features.shape[0], sqlen,sqlen , 3)  # Reshape all images in one go
    return features

def extractest_features(images):
    features = []
    for image in tqdm(images):
      try:
        img = load_img(image, target_size=(sqlen, sqlen))
        img = np.array(img)
        features.append(img)

      except Exception as e:
        print(image)
        temp_dir.append(image)
        continue

    for i in (temp_dir):
       test_paths.remove(i)

    features = np.array(features)
    features = features.reshape(features.shape[0], sqlen, sqlen, 3)  # Reshape all images in one go
    return features

TRAIN_DIR = "/content/drive/MyDrive/New_Data/Train"

train = pd.DataFrame()
train['image'], train['label'] = createdataframe(TRAIN_DIR)

train_features = extractrain_features(train['image'])

x_train = train_features / 255.0

le = LabelEncoder()
le.fit(train['label'])
y_train = le.transform(train['label'])
y_train = to_categorical(y_train, num_classes=2)



model = Sequential()


model.add(Conv2D(32, kernel_size=(3, 3), activation='relu',input_shape=(sqlen, sqlen, 3), padding='same'))
model.add(AveragePooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(AveragePooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(512, activation='relu'))

model.add(Dense(64, activation='relu'))

model.add(Dense(2, activation='softmax'))

# Compile model
model.compile(optimizer='adam', loss=tf.keras.losses.Huber(delta=1.0), metrics=['mae'])
# Fit model
model.fit(x=x_train,y=y_train, batch_size=20, epochs=20)

TEST_DIR = "/content/drive/MyDrive/induction-task-2025/Test_Images"
test_images = sorted([img for img in os.listdir(TEST_DIR)],
                     key=lambda x: int(x.split('_')[1].split('.')[0]))
test_paths = [os.path.join(TEST_DIR, img) for img in test_images]

x_test = extractest_features(test_paths)
x_test = x_test / 255.0  # Normalize test data



predictions = model.predict(x_test)
predicted_labels = le.inverse_transform(np.argmax(predictions, axis=1))



submission_data = []
image_to_label = dict(zip(test_paths, predicted_labels))

for img_path in test_paths:
    img_name = os.path.basename(img_path).split('.')[0]  # Remove the file extension
    submission_data.append((img_name, image_to_label[img_path]))

print("Creating submission file...")
submission = pd.DataFrame(submission_data, columns=['Id', 'Label'])
submission.to_csv("submitit.csv", index=False)
print("Submission file created successfully!")

In [ ]:
import csv

# Data you want to append to the CSV file
default_data = []

for i in (temp_dir):
 new_i = os.path.basename(i).split('.')[0]
 default_data.append([new_i , "Real"])

# Path to your CSV file
csv_file_path = '/content/submito.csv'

# Open the CSV file in append mode
with open(csv_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)

    # Append the new data as a row in the CSV
    for j in default_data:
     writer.writerow(j)
